## Copy files, you will label, in a separate directroy for convinience

In [103]:
import os
import shutil


source_directory = r"D:\images\images"
destination_directory = r"D:\images\rimages"  # replace with the path to your destination directory


files = os.listdir(source_directory)
files = sorted(files)

In [104]:
len(files)

117022

### Specify indexes of files you will label by changing COPY_FROM, COPY_TO values 

In [105]:
COPY_FROM = 0           # <--------------
COPY_TO = 2000          # <--------------

if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)

your_files = files[COPY_FROM: COPY_TO]

for file in your_files:
    source_file = os.path.join(source_directory, file)
    destination_file = os.path.join(destination_directory, file)
    shutil.copy2(source_file, destination_file)

### Create Excel file with you labels

In [ ]:
import pandas as pd

labeling_file = 'image_labeling.xlsx'

pd.DataFrame(
    {
        'image_path': your_files,
        'is_interior': [0 for _ in your_files]
    }
).to_excel('image_labeling.xlsx', index=False)

In [107]:
labels = pd.read_excel(labeling_file)


In [108]:
labels['is_interior'].value_counts()

0    1408
1     592
Name: is_interior, dtype: int64

### Train test split (stratified)

In [109]:
import math

train_size_food = math.ceil(.75 * len(labels[labels['is_interior'] == 0]))
train_size_interior = math.ceil(.75 * len(labels[labels['is_interior'] == 1]))

train_food_file_paths = labels[labels['is_interior'] == 0].head(train_size_food)['image_path']
test_food_file_paths = list(
    set(labels[labels['is_interior'] == 0]['image_path']) - 
    set(train_food_file_paths)
)


train_interior_file_paths = labels[labels['is_interior'] == 1].head(train_size_interior)['image_path']
test_interior_file_paths = list(
    set(labels[labels['is_interior'] == 1]['image_path']) - 
    set(train_interior_file_paths)
)

In [110]:
len(train_food_file_paths) + len(test_food_file_paths) + len(train_interior_file_paths) + len(test_interior_file_paths)

2000

In [115]:
data_directory = r"D:\labeled_images"            # <----------- Directory where your train test data will be stored
if not os.path.exists(destination_directory):
    os.makedirs(destination_directory)
    

train_food_dir = os.path.join(data_directory, 'train', 'food')
test_food_dir = os.path.join(data_directory, 'test', 'food')

train_interior_dir = os.path.join(data_directory, 'train', 'interior')
test_interior_dir = os.path.join(data_directory, 'test', 'interior')


for directory in [train_food_dir, test_food_dir, train_interior_dir, test_interior_dir]:
    if not os.path.exists(directory):
        os.makedirs(directory)
        
file_map = {
    train_food_dir: train_food_file_paths,
    test_food_dir: test_food_file_paths,
    train_interior_dir: train_interior_file_paths,
    test_interior_dir: test_interior_file_paths
}


for image_class_dir, file_paths in file_map.items():
    for file in file_paths:
        source_file = os.path.join(source_directory, file)
        destination_file = os.path.join(image_class_dir, file)
        shutil.copy2(source_file, destination_file)